# Decompose MOB data

Apply NMF or sICA factorization to MOB data

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.curdir))

In [ ]:
from configobj import ConfigObj
import os, glob
import numpy as np
from regnmf import ImageAnalysisComponents as ia
from scipy.spatial.distance import pdist
import matplotlib.pyplot as plt
from PIL import Image

### Specify parameter

In [ ]:
basepath = os.path.abspath(os.path.join(os.path.pardir, "Soelter_et_al_raw_data"))
#basepath = os.path.join('/media/jan/BackupWork/Documents/NewAnalysis')
datapath = os.path.join(basepath, 'MOBconverted') #where to get the data
savepath = os.path.join(basepath, 'MOBdecomposed') #where to save decompostion
savepath_vis = os.path.join(basepath, "Vis","Factorizations") #where to save visualization of decomposition
cfgfile = os.path.join(basepath, 'configfiles', 'decompose', 'nnmf_20_sm5_convex_negTimelowSP.ini') #'sica_200.ini') #
datafile = 'sph_meas'#'ios_meas'
response_window = (8,12) #define frames to calculate odor response; ios:(3,5) sph:(8,12)

In [ ]:
animals =  [ani for ani in os.listdir(datapath) if ('FRV' in ani)] #('FRV' not in ani)]

### NMF factorization

Performs NMF factorization according to config file. The sparsness parameter is choosen iterativly: First start with 'sparse_start' as initial guess. Sparseness will increase in steps of 'sparse_increase' until spatial component correlation of stimulus dependent components drops below 0.5

In [ ]:
sparse_start = 0.1 #inital sparseness strength
sparse_increase = 0.1 #sparsness increase
redo = False# redo factorization if it already exists
assert 'nnmf' in cfgfile

for animal in animals:
    
    #load config
    cfg = ConfigObj(cfgfile, unrepr=True)
    
    #check if computation is already performed
    savelocation = os.path.join(savepath, animal)
    savename_mask = os.path.splitext(os.path.basename(cfgfile))[0] + '_sp*_' + datafile + '.npy'
    if os.path.exists(savelocation):
        if len(glob.glob(os.path.join(savelocation, savename_mask)))>0 and not(redo):
            print '%s already done'%animal
            continue
    else:
        os.makedirs(savelocation)
    
    #load data
    filename = os.path.join(datapath, animal, datafile)
    ts = ia.TimeSeries()
    try:
        ts.load(filename)
    except IOError:
        print '!!! No data for animal %s !!!'%animal
        continue
    
    # perform decomposition. increase sparseness until spatial component correlation is below 0.5
    cor = 1
    while cor>0.5:
        # set sparsness level
        if 'sparse_param' in cfg:
            cfg['sparse_param'] += sparse_increase
        else:
            cfg['sparse_param'] = sparse_start
        
        # perform decomposition
        decomposer = ia.NNMF(**cfg)
        decomposition = decomposer(ts)
        decomposition.base._series[np.isnan(decomposition.base._series)] = 0 #clear nans
            
        # calc spatial cor of stimulus driven components
        signal = ia.TrialMean()(ia.CutOut(response_window)(decomposition))
        mode_cor = ia.CalcStimulusDrive()(signal)
        mask = mode_cor._series.squeeze()<0.5
        if np.sum(mask) > 1: #if there are stimulus driven components  
            selected_modes = ia.SelectObjects()(decomposition, mask)   
            cor = np.nanmax(1-pdist(selected_modes.base._series, 'correlation'))
        else:
            cor = np.nanmax(1-pdist(decomposition.base._series, 'correlation'))
    
    #save results
    savename = os.path.splitext(os.path.basename(cfgfile))[0] + '_sp%02d'%(cfg['sparse_param']*10)+'_'+datafile
    decomposition.save(os.path.join(savelocation, savename))
    print '%s done'%animal

In [ ]:
np.sum(mask)

### sICA factorization

In [ ]:
redo = True
assert 'sica' in cfgfile

for animal in animals:
    
    savelocation = os.path.join(savepath, animal)
    savename = os.path.splitext(os.path.basename(cfgfile))[0] + '_' + datafile
    if os.path.exists(savelocation):
        if os.path.exists(os.path.join(savelocation, savename + '.npy')) and not(redo):
            print '%s already done'%animal
            continue
    else:
        os.makedirs(savelocation)
        
    filename = os.path.join(datapath, animal, datafile)
    ts = ia.TimeSeries()
    try:
        ts.load(filename)
    except IOError:
        print '!!! No data for animal %s !!!'%animal
        continue
    cfg = ConfigObj(cfgfile, unrepr=True)
        
    # perform decomposition
    decomposer = ia.sICA(**cfg)
    decomposition = decomposer(ts)
    decomposition.base._series[np.isnan(decomposition.base._series)] = 0 #clear nans
   
    decomposition.save(os.path.join(savelocation, savename))
    print '%s done'%animal

### Plot decomposition overview

Plot decompositions for selected animals

In [ ]:
method = 'sica_200_ios_meas'#'nnmf_200_sm2_convex_negTimelowSP_sp*_ios_meas'
draw_sphrois = True
animals = [ani for ani in os.listdir(savepath) if ('FRV' not in ani)]

In [ ]:
rows = np.ceil(len(animals)/5.)
fig = plt.figure(figsize=(10, 1.5*int(rows)))
plt.subplots_adjust(left=0.02, bottom=0.02, right=0.98, top=0.9)

for ix, animal in enumerate(animals):
    ax = fig.add_subplot(rows, 5, ix+1)
    bg = Image.open(os.path.join(datapath, animal, 'bg.png'))
    bg = bg.convert('L')
    
    mf = ia.TimeSeries()
    filename = glob.glob(os.path.join(savepath, animal, method+'.npy'))
    assert len(filename) == 1
    filename = filename[0].split('.')[0]
    mf.load(filename)
    
    bg = bg.resize(mf.base.shape[::-1])
    bg = np.asarray(bg)
    if mf.name.split('_')[1] == 'l':
        bg = bg[::-1]    
       
    mf = ia.TrialMean()(ia.CutOut(response_window)(mf))
    t2t = ia.CalcStimulusDrive()(mf)._series.squeeze()
    
    myextent = np.array([0, mf.base.shape[1], mf.base.shape[0], 0])-0.5
    ax.imshow(bg, interpolation='none', extent=myextent, cmap=plt.cm.bone)
    for ix, base in enumerate(mf.base.shaped2D()):
        mycolors = ['c', 'b'] if t2t[ix] <0.4 else ['0.3', '0.7']
        if mf.name.split('_')[1] == 'l':
            base  = base[::-1]    
        ax.contourf(base, [0.3,0.7,1], colors=mycolors, alpha=0.5)
    
    #show rois
    roi_path = os.path.join(datapath, animal, 'rois')
    if draw_sphrois and os.path.exists(roi_path+'.npy'):
        rois = ia.TimeSeries()
        rois.load(roi_path)
        for roi in rois.shaped2D():
            if mf.name.split('_')[1] == 'l':
                roi = roi[::-1]
            ax.contour(roi, [0.5], colors=['w'], lw=2)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(animal, size=8)
    
fig.savefig(os.path.join(savepath_vis, method+'.pdf'))
plt.show()